# 3ML with Fermi GBM TTE  and LAT LLE Data
### Purpose

This demonstrates of you can use LLE data directly in 3ML

For more detail on the Fermi TTE plugin, check out its demo.

#### Let's check it out!

Import **3ML** as always to make sure you have the plugin.

In [1]:
%matplotlib inline
%matplotlib notebook
import numpy as np
from threeML import *

get_available_plugins()

Configuration read from /Users/jburgess/.threeML/threeML_config.yml


ImportError: cannot import name cleanup_downloaded_LLE_datapy

In [ ]:
# os.path.join is a way to generate system-independent
# paths (good for unix, windows, Mac...)
trigger_number = 'bn080916009'

data_dir_gbm = os.path.join('gbm',trigger_number)
gbm_data = download_GBM_trigger_data(trigger_number,detectors=['n3','n4','b0'],destination_directory=data_dir_gbm,compress_tte=True)


data_dir_lle = os.path.join('lat','lle',trigger_number)

lle_data = download_LLE_trigger_data(trigger=trigger_number,destination_directory=data_dir_lle)

src_selection = "0.-71."


# We start out with a bad background interval to demonstrate a few features

nai3 = FermiGBMTTELike('NAI3',
                         os.path.join(data_dir_gbm, "glg_tte_n3_bn080916009_v01.fit.gz"),
                         "-10-0,100-200",
                         src_selection,
                         rsp_file=os.path.join(data_dir_gbm, "glg_cspec_n3_bn080916009_v00.rsp2"),poly_order=-1,verbose=False)

nai4 = FermiGBMTTELike('NAI4',
                         os.path.join(data_dir_gbm, "glg_tte_n4_bn080916009_v01.fit.gz"),
                         "-10-0,100-200",
                         src_selection,
                         rsp_file=os.path.join(data_dir_gbm, "glg_cspec_n4_bn080916009_v00.rsp2"),poly_order=-1,verbose=False)

bgo0 = FermiGBMTTELike('BGO0',
                         os.path.join(data_dir_gbm, "glg_tte_b0_bn080916009_v01.fit.gz"),
                         "-10-0,100-200",
                         src_selection,
                         rsp_file=os.path.join(data_dir_gbm, "glg_cspec_b0_bn080916009_v00.rsp2"))


lle = FermiLATLLELike("LLE",
                      os.path.join(data_dir_lle, "gll_lle_bn080916009_v10.fit"),
                      os.path.join(data_dir_lle, "gll_pt_bn080916009_v10.fit"),
                      "-100-0,100-200",
                      src_selection,
                      rsp_file=os.path.join(data_dir_lle, "gll_cspec_bn080916009_v10.rsp"),poly_order=-1)

In [ ]:
lle.peek()

Let's look at the lightcurve of NAI3 to check out background fit:

In [ ]:
nai3.view_lightcurve(-10,100.,.5)

In [ ]:
lle.view_lightcurve(-10,100,dt=.5)

### Energy selection



In [ ]:
nai3.set_active_measurements("10.0-30.0", "40.0-900.0")
nai3.view_count_spectrum()

In [ ]:
nai4.set_active_measurements("10.0-30.0", "40.0-900.0")
nai4.view_count_spectrum()

In [ ]:
bgo0.set_active_measurements("250-43000")
bgo0.view_count_spectrum()

In [ ]:
lle.set_active_measurements("50000-100000")
lle.view_count_spectrum()

## Fitting!

We are now ready for the standard **3ML** process:


In [ ]:

ra = 121.8
dec = -61.3

data_list = DataList(nai3,nai4,bgo0,lle )

band= Band()

GRB = PointSource( trigger_number, ra, dec, spectral_shape=band )

model = Model( GRB )


In [ ]:
jl = JointLikelihood( model, data_list, verbose=False )

res = jl.fit()

In [ ]:
sp = SpectralPlotter(jl)

_=sp.plot_model(y_unit='erg2/(cm2 s keV)', num_ene=200,x_max=1E5)

We can examine our fit with the data:

In [ ]:
_ = display_ogip_model_counts(jl,min_rate=1,step=False)

In [ ]:
res = jl.get_errors()

In [ ]:
res = jl.get_contours(band.xp,200,900,20)

In [ ]:
res = jl.get_contours(band.xp,250,900,50,band.alpha,-1.2,-0.9,50)